## Prioritization for data cleaning

- We should focus on cleaning up the metrics first and do some feature engineering on these to make sure they are in a state that's valuable to FuF
  - Sidewalk damage: HARDSCAPE
  - Size of trees: DBH
  - Density of trees: Lat/Long - create additional metric
  - Neighborhood: PROPERTY
  - Health of tree: CONDITION - definitely needs cleaning, could simplify categories
  - Types of tree: (Genus and Species): BOTANICAL - may be useful to seperate out genus and species here; Common name: COMMON

## Known issues / areas to work on

- ON_ADR/ONSTREET and PROP_ADR/PROPSTREET are the same except for in some cases, we think it may be because of street corners?
- ON_ADR/ONSTREET and PROP_ADR/PROPSTREET need to be merged to produce addresses
- BOTANICAL needs to be split apart into GENUS and Species
- Many categorical variables, especially the metrics, have redundant/messy categorical values that need to be cleaned up.
- We can create binary indicators from many of these categorical variables and aggregate them to create proportions at the neighborhood level, which would be much easier to visualize on the map, and could also then be exported for decision-making purposes.


Look at condition store by species and neighborhood, and look at condition disparity 

In [28]:
import pandas as pd, numpy as np

fuf_data = pd.read_excel('../data/fuf_consolidated_thru_8_22.xlsx', sheetname=0)
fuf_data_updated = pd.read_csv('../data/combined_tree_data_with_header.csv', header = False)

In [30]:
fuf_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83887 entries, 14243 to 98129
Data columns (total 28 columns):
BOTANICAL     68916 non-null object
CLEARANCE     46303 non-null object
COMMON        68917 non-null object
CONDITION     68917 non-null object
DBH           63950 non-null object
EXACT_DBH     56650 non-null float64
Exact_DBH     14967 non-null float64
HARDSCAPE     62858 non-null object
ID            83887 non-null int64
Lat           68917 non-null float64
Long          68917 non-null float64
NOTES         2451 non-null object
OBJECTID      14243 non-null float64
OBSERVATIO    52614 non-null object
ONSTREET      68905 non-null object
ON_ADR        68917 non-null float64
PARKWAY_SI    68917 non-null float64
PARKWAY_TY    68917 non-null object
PROPERTY      68917 non-null object
PROPSTREET    62005 non-null object
PROP_ADR      68917 non-null float64
RECMAINT      68916 non-null object
SIDE          68906 non-null object
SITE          68917 non-null float64
STATUS        68

In [32]:
fuf_data_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112331 entries, 0 to 112330
Data columns (total 18 columns):
tree_id                      112331 non-null object
neighborhood                 112331 non-null object
on_street_name               112320 non-null object
side                         107443 non-null object
scientific_species_name      112330 non-null object
common_species_name          112279 non-null object
diameter_at_breast_height    112331 non-null int64
condition                    108936 non-null object
parkway_space_type           112302 non-null object
parkway_largest_dimension    112331 non-null int64
trunks                       112331 non-null int64
maintenance_notes            112330 non-null object
status                       75304 non-null float64
hardscape_damage             106101 non-null object
observation_notes            100767 non-null object
clearance                    89077 non-null object
longitude                    112331 non-null float64
latitude

- Only on_street_name now, FuF doesnt care too much about the address

In [31]:
data = fuf_data[['ON_ADR','ONSTREET', 'PROPSTREET', 'PROP_ADR']]
data.count()

ON_ADR        68917
ONSTREET      68905
PROPSTREET    62005
PROP_ADR      68917
dtype: int64

In [34]:
data = fuf_data[['ON_ADR','ONSTREET', 'PROPSTREET', 'PROP_ADR']]
data.ix[data['ONSTREET'] != data['PROPSTREET']].head()

,ON_ADR,ONSTREET,PROPSTREET,PROP_ADR
14337,1698,17TH AVE,MORAGA ST,1050
14365,1498,17TH AVE,KIRKHAM ST,1252
14366,1498,17TH AVE,KIRKHAM ST,1252
14367,1498,17TH AVE,KIRKHAM ST,1252
14368,1498,17TH AVE,KIRKHAM ST,1252


In [13]:
fuf_data.describe()

,EXACT_DBH,Exact_DBH,ID,Lat,Long,OBJECTID,ON_ADR,PARKWAY_SI,PROP_ADR,SITE,STATUS,TRUNKS,UTILITY
count,56650.000000,14967.000000,83887.000000,68917.000000,68917.000000,14243.000000,68917.000000,68917.000000,68917.000000,68917.000000,68917.000000,68917.000000,68917
mean,8.708597,7.171377,692119.092923,37.752141,-122.432734,22356.554027,1068.098176,6.638899,1034.382141,3.364496,1.150906,1.075177,0
std,10.760350,6.928085,170706.449926,0.019591,0.020166,5440.171394,1187.553663,16.024663,1209.596622,6.975026,0.521695,0.773977,0
min,0.000000,0.000000,200610.000000,37.708306,-122.478378,12559.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0
25%,4.000000,2.000000,601485.500000,37.737962,-122.445896,17702.500000,200.000000,3.000000,120.000000,1.000000,1.000000,1.000000,0
50%,7.000000,6.000000,715067.000000,37.751631,-122.432168,22147.000000,600.000000,3.000000,540.000000,1.000000,1.000000,1.000000,0
75%,12.000000,11.000000,811509.500000,37.762731,-122.418838,26956.500000,1485.000000,4.000000,1474.000000,3.000000,1.000000,1.000000,0
max,1920.000000,74.000000,912780.000000,37.806670,-122.382925,32287.000000,33369.000000,99.000000,33369.000000,122.000000,6.000000,27.000000,0


- Exact_DBH != Exact_DBH - could be a join issue. 	EXACT_DBH looks like the more complete field

In [14]:
fuf_data['HARDSCAPE'].describe()

count     62858
unique       10
top        None
freq      42856
Name: HARDSCAPE, dtype: object

In [15]:
fuf_data['HARDSCAPE'].value_counts()

None                  42856
Sidewalk/CG           11012
Sidewalk               7381
Curb/Gutter            1355
Temporary               144
Other                    52
Private                  29
Well Grate / Cover       27
Temporary/CG              1
0                         1
dtype: int64

In [10]:
fuf_data['PROPERTY'].value_counts()

Noe Valley             8841
Mission District       8307
West of Twin Peaks     7928
Bernal Heights         5448
Potrero Hill           5376
Castro/Upper Market    5228
Outer Mission          4819
Excelsior              3759
Inner Sunset           3718
Pacific Heights        2345
Presidio Heights       2314
Glen Park              2217
Western Addition       1876
Nob Hill               1522
Haight-Ashbury         1261
Russian Hill           1071
Visitacion Valley       845
Parkside                727
Crocker Amazon          712
Diamond Heights         302
South of Market         179
Twin Peaks               78
Inner Richmond           44
dtype: int64

In [38]:
fuf_data['CONDITION'].value_counts()

Fair               27400
Poor               24958
Good                9514
Vacancy             4834
Dead                 695
Very Good            588
Stump                357
Excellent            158
Stump Removal        150
Critical             118
Open                 109
Unsuitable Site       35
6/15/2016              1
dtype: int64

In [15]:
fuf_data['HARDSCAPE_BINARY'] = fuf_data['HARDSCAPE'].apply(lambda x: 0 if x == 'None' else 1)
fuf_data['VACANT_LOT'] = fuf_data['CONDITION'].apply(lambda x: 0 if x != 'Vacancy' else 1)
fuf_data['CONDITION_BINARY'] = fuf_data['CONDITION'].apply(lambda x: 0 if x in ['Poor','Dead',
                                                                               'Critical'] else 1)
fuf_data['STUMP'] = fuf_data['CONDITION'].apply(lambda x: 1 if x in ['Stump', 'Stump Removal'] else 0)
grouped = fuf_data[['HARDSCAPE_BINARY','EXACT_DBH','PROPERTY',
          'VACANT_LOT', 'CONDITION_BINARY', 'STUMP']].groupby('PROPERTY').mean().reset_index()
joined = pd.merge(fuf_data, grouped, on='PROPERTY', how = 'left', suffixes=('_binary','_perc'))

In [16]:
grouped

,PROPERTY,HARDSCAPE_BINARY,EXACT_DBH,VACANT_LOT,CONDITION_BINARY,STUMP
0,Bernal Heights,0.448421,9.521822,0.055800,0.599853,0.005690
1,Castro/Upper Market,0.428462,9.754973,0.024292,0.392119,0.011668
2,Crocker Amazon,0.669944,6.137640,0.179775,0.706461,0.009831
3,Diamond Heights,0.069536,22.000000,0.033113,0.291391,0.006623
4,Excelsior,0.585794,7.486034,0.048151,0.613727,0.007449
5,Glen Park,0.461885,10.618854,0.046910,0.496617,0.014885
6,Haight-Ashbury,0.390167,9.438443,0.038065,0.827121,0.000793
7,Inner Richmond,0.772727,NaN,0.000000,0.136364,0.000000
8,Inner Sunset,0.362829,7.885511,0.075578,0.838892,0.008876
9,Mission District,0.377272,8.095176,0.047791,0.631154,0.009149


In [197]:
joined.columns

Index([              u'BOTANICAL',               u'CLEARANCE',
                        u'COMMON',               u'CONDITION',
                           u'DBH',        u'EXACT_DBH_binary',
                     u'Exact_DBH',               u'HARDSCAPE',
                            u'ID',                     u'Lat',
                          u'Long',                   u'NOTES',
                      u'OBJECTID',              u'OBSERVATIO',
                      u'ONSTREET',                  u'ON_ADR',
                    u'PARKWAY_SI',              u'PARKWAY_TY',
                      u'PROPERTY',              u'PROPSTREET',
                      u'PROP_ADR',                u'RECMAINT',
                          u'SIDE',                    u'SITE',
                        u'STATUS',                  u'TRUNKS',
                       u'UTILITY',                u'filename',
       u'HARDSCAPE_BINARY_binary',       u'VACANT_LOT_binary',
       u'CONDITION_BINARY_binary',         u'UNSUITABLE

In [10]:
joined.head()

,BOTANICAL,CLEARANCE,COMMON,CONDITION,DBH,EXACT_DBH_binary,Exact_DBH,HARDSCAPE,ID,Lat,...,filename,HARDSCAPE_BINARY_binary,VACANT_LOT_binary,CONDITION_BINARY_binary,STUMP_binary,HARDSCAPE_BINARY_perc,EXACT_DBH_perc,VACANT_LOT_perc,CONDITION_BINARY_perc,STUMP_perc
0,Tristaniopsis laurina,None,Water Gum,Good,00-03,3,NaN,None,200610,37.781515,...,5_31_rest_of_all_WGS84.xlsx,0,0,1,0,0.171564,10.043896,0.08427,0.712619,0.006914
1,Stump,None,Stump,Stump,07-12,7,NaN,None,200611,37.781501,...,5_31_rest_of_all_WGS84.xlsx,0,0,1,1,0.171564,10.043896,0.08427,0.712619,0.006914
2,Prunus x blireiana,None,Double-Flowering Plum,Fair,04-06,6,NaN,None,200612,37.781497,...,5_31_rest_of_all_WGS84.xlsx,0,0,1,0,0.171564,10.043896,0.08427,0.712619,0.006914
3,Metrosideros excelsa,None,New Zealand Christmas Tree,Fair,07-12,11,NaN,None,200614,37.781487,...,5_31_rest_of_all_WGS84.xlsx,0,0,1,0,0.171564,10.043896,0.08427,0.712619,0.006914
4,Metrosideros excelsa,None,New Zealand Christmas Tree,Good,07-12,12,NaN,None,200615,37.781484,...,5_31_rest_of_all_WGS84.xlsx,0,0,1,0,0.171564,10.043896,0.08427,0.712619,0.006914


In [27]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83887 entries, 0 to 83886
Data columns (total 37 columns):
BOTANICAL                  68916 non-null object
CLEARANCE                  46303 non-null object
COMMON                     68917 non-null object
CONDITION                  68917 non-null object
DBH                        63950 non-null object
EXACT_DBH_binary           56650 non-null float64
Exact_DBH                  14967 non-null float64
HARDSCAPE                  62858 non-null object
ID                         83887 non-null int64
Lat                        68917 non-null float64
Long                       68917 non-null float64
NOTES                      2451 non-null object
OBJECTID                   14243 non-null float64
OBSERVATIO                 52614 non-null object
ONSTREET                   68905 non-null object
ON_ADR                     68917 non-null float64
PARKWAY_SI                 68917 non-null float64
PARKWAY_TY                 68917 non-null object
PROP

In [21]:
joined.to_csv("fuf_with_metrics.csv")